In [1]:
import torch
import torch.nn as nn

### A (very small) introduction to pytorch

Pytorch Tensors are very similar to Numpy arrays, with the added benefit of being usable on GPU. For a short tutorial on various methods to create tensors of particular types, see [this link](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py).
The important things to note are that Tensors can be created empty, from lists, and it is very easy to convert a numpy array into a pytorch tensor, and inversely.

In [2]:
a = torch.LongTensor(5)
b = torch.LongTensor([5])

print(a)
print(b)

tensor([140574092781552, 140574091624176, 140574093475312, 140574093475184,
        140574017510288])
tensor([5])


In [3]:
a = torch.FloatTensor([2])
b = torch.FloatTensor([3])

print(a + b)

tensor([5.])


The main interest in us using Pytorch is the ```autograd``` package. ```torch.Tensor```objects have an attribute ```.requires_grad```; if set as True, it starts to track all operations on it. When you finish your computation, can call ```.backward()``` and all the gradients are computed automatically (and stored in the ```.grad``` attribute).

One way to easily cut a tensor from the computational once it is not needed anymore is to use ```.detach()```.
More info on automatic differentiation in pytorch on [this link](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py).

In [4]:
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


In [5]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
for name, p in linear.named_parameters():
    print(name)
    print(p)

# Build loss function - Mean Square Error
criterion = nn.MSELoss()

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('Initial loss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

weight
Parameter containing:
tensor([[-0.4873, -0.5195,  0.4941],
        [-0.1611,  0.2527,  0.4156]], requires_grad=True)
bias
Parameter containing:
tensor([0.3051, 0.2095], requires_grad=True)
Initial loss:  1.3340483903884888
dL/dw:  tensor([[-0.2969,  0.1286,  1.1893],
        [-0.0129, -0.1413,  0.6312]])
dL/db:  tensor([0.1414, 0.4640])


In [6]:
# You can perform gradient descent manually, with an in-place update ...
linear.weight.data.sub_(0.01 * linear.weight.grad.data)
linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('Loss after one update: ', loss.item())

Loss after one update:  1.3125115633010864


In [7]:
# Use the optim package to define an Optimizer that will update the weights of the model.
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# By default, gradients are accumulated in buffers( i.e, not overwritten) whenever .backward()
# is called. Before the backward pass, we need to use the optimizer object to zero all of the
# gradients.
optimizer.zero_grad()
loss.backward()

# Calling the step function on an Optimizer makes an update to its parameters
optimizer.step()

# Print out the loss after the second step of gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('Loss after two updates: ', loss.item())

Loss after two updates:  1.2917243242263794


### Tools for data processing 

In [8]:
import os
import time
import math
from collections import Counter
import pprint
pp = pprint.PrettyPrinter(indent=1)

We create a ```Dictionary``` class, that we are going to use to create a vocabulary for our text data. The goal here is to have a convenient tool, with easy access to any information we could need:
- A python dictionary ```word2idx``` allowing easy transformation of tokenized text into indexes
- A list ```idx2word```, allowing us to find the word corresponding to an index (for interpretation and generation)
- A python dictionary ```counter``` used to build the vocabulary, that can provide us with frequency information if needed. 
- The ```total``` count of words in the dictionary.

Important: The data that we are going to use are already pre-processed so we don't need to create special tokens and control the size of the vocabulary ourselves. However, when the text data is raw, methods to preprocess it conveniently should be added here. 

In [9]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.counter = {}
        self.total = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
            self.counter.setdefault(word, 0)
        self.counter[word] += 1
        self.total += 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

In [10]:
with open('./wikitext-2/train.txt', 'r', encoding='utf-8') as f:
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())

 

 = Valkyria Chronicles III = 

 

 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 



In [11]:
# Let's take the four first lines of our training data:
corpus = ''
with open('./wikitext-2/train.txt', 'r') as f:
    for i in range(4):
        corpus += f.readline()
        
# Create an empty Dictionary, separate and add all words. 
dictio = Dictionary()
words = corpus.split()
for word in words:
    dictio.add_word(word)

# Take a look at the objects created:
pp.pprint(dictio.word2idx)
pp.pprint(dictio.idx2word)
pp.pprint(dictio.counter)
pp.pprint(dictio.total)

{'"': 60,
 '(': 9,
 ')': 18,
 ',': 12,
 '.': 14,
 '2011': 44,
 '3': 6,
 ':': 7,
 '<unk>': 8,
 '=': 0,
 '@-@': 29,
 'Battlefield': 17,
 'Chronicles': 2,
 'Europan': 70,
 'Gallia': 67,
 'III': 3,
 'Imperial': 80,
 'January': 43,
 'Japan': 24,
 'Japanese': 10,
 'Media.Vision': 37,
 'Nameless': 61,
 'PlayStation': 39,
 'Portable': 40,
 'Raven': 81,
 'Released': 41,
 'Second': 69,
 'Sega': 35,
 'Senjō': 4,
 'Valkyria': 1,
 'War': 71,
 'a': 26,
 'against': 79,
 'and': 36,
 'are': 77,
 'as': 22,
 'black': 75,
 'by': 34,
 'commonly': 19,
 'developed': 33,
 'during': 68,
 'first': 58,
 'follows': 59,
 'for': 38,
 'fusion': 49,
 'game': 32,
 'gameplay': 52,
 'in': 42,
 'is': 25,
 'it': 45,
 'its': 53,
 'lit': 13,
 'military': 63,
 'nation': 66,
 'no': 5,
 'of': 15,
 'operations': 76,
 'outside': 23,
 'parallel': 57,
 'penal': 62,
 'perform': 73,
 'pitted': 78,
 'playing': 30,
 'predecessors': 54,
 'real': 50,
 'referred': 20,
 'role': 28,
 'runs': 56,
 'same': 48,
 'secret': 74,
 'series': 47,
 

In [12]:
class Corpus(object):
    def __init__(self, path):
        # We create an object Dictionary associated to Corpus
        self.dictionary = Dictionary()
        # We go through all files, adding all words to the dictionary
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))
        
    def tokenize(self, path):
        """Tokenizes a text file, knowing the dictionary, in order to tranform it into a list of indexes"""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)
                tokens += len(words)
        
        # Once done, go through the file a second time and fill a Torch Tensor with the associated indexes 
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        return ids

In [14]:
###############################################################################
# Load data
###############################################################################

data = './wikitext-2/'
corpus = Corpus(data)

In [15]:
print(corpus.dictionary.total)
print(len(corpus.dictionary.idx2word))
print(len(corpus.dictionary.word2idx))

print(corpus.train.shape)
print(corpus.train[0:7])
print([corpus.dictionary.idx2word[corpus.train[i]] for i in range(7)])

print(corpus.valid.shape)
print(corpus.valid[0:7])
print([corpus.dictionary.idx2word[corpus.valid[i]] for i in range(7)])

2551843
33278
33278
torch.Size([2088628])
tensor([0, 1, 2, 3, 4, 1, 0])
['<eos>', '=', 'Valkyria', 'Chronicles', 'III', '=', '<eos>']
torch.Size([217646])
tensor([    0,     1, 32966, 32967,     1,     0,     0])
['<eos>', '=', 'Homarus', 'gammarus', '=', '<eos>', '<eos>']


In [16]:
# We now have data under a very long list of indexes: the text is as one sequence.
# The idea now is to create batches from this. Note that this is absolutely not the best
# way to proceed with large quantities of data (where we'll try not to store huge tensors
# in memory but read them from file as we go) !
# Here, we are looking for simplicity and efficiency with regards to computation time.
# That is why we will ignore sentence separations and treat the data as one long stream that
# we will cut arbitrarily as we need.
# With the alphabet being our data, we currently have the sequence:
# [a b c d e f g h i j k l m n o p q r s t u v w x y z]
# We want to reorganize it as independant batches that will be processed independantly by the model !
# For instance, with the alphabet as the sequence and batch size 4, we'd get the 4 following sequences:
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘
# with the last two elements being lost.
# Again, these columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient processing.

def batchify(data, batch_size, cuda = False):
    # Cut the elements that are unnecessary
    nbatch = data.size(0) // batch_size
    data = data.narrow(0, 0, nbatch * batch_size)
    # Reorganize the data
    data = data.view(batch_size, -1).t().contiguous()
    # If we can use a GPU, let's tranfer the tensor to it
    if cuda:
        data = data.cuda()
    return data

# get_batch subdivides the source data into chunks of the appropriate length.
# If source is equal to the example output of the batchify function, with
# a sequence length (seq_len) of 3, we'd get the following two variables:
# ┌ a g m s ┐ ┌ b h n t ┐
# | b h n t | | c i o u │
# └ c i o u ┘ └ d j p v ┘
# The first variable contains the letters input to the network, while the second
# contains the one we want the network to predict (b for a, h for g, v for u, etc..)
# Note that despite the name of the function, we are cutting the data in the
# temporal dimension, since we already divided data into batches in the previous
# function. 

def get_batch(source, i, seq_len, evaluation=False):
    # Deal with the possibility that there's not enough data left for a full sequence
    seq_len = min(seq_len, len(source) - 1 - i)
    # Take the input data
    data = source[i:i+seq_len]
    # Shift by one for the target data
    target = source[i+1:i+1+seq_len]
    return data, target

In [17]:
batch_size = 100
eval_batch_size = 4
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

print(train_data.shape)
print(val_data.shape)

torch.Size([20886, 100])
torch.Size([54411, 4])


In [18]:
input_words, target_words = get_batch(val_data, 0, 3)
pp.pprint(input_words)
pp.pprint(target_words)
input_words, target_words = get_batch(val_data, 3, 3)
pp.pprint(input_words)
pp.pprint(target_words)

tensor([[    0,  1839,   284,   877],
        [    1,     9,   613,    17],
        [32966,     9,  2123, 11033]])
tensor([[    1,     9,   613,    17],
        [32966,     9,  2123, 11033],
        [32967,     9,    13,  4548]])
tensor([[32967,     9,    13,  4548],
        [    1,     9,    35,    16],
        [    0,     9, 29595,  2582]])
tensor([[    1,     9,    35,    16],
        [    0,     9, 29595,  2582],
        [    0,    10,    13,    37]])


### LSTM Cells in pytorch

In [19]:
# Create a toy example of LSTM: 
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# LSTMs expect inputs having 3 dimensions:
# - The first dimension is the temporal dimension, along which we (in our case) have the different words
# - The second dimension is the batch dimension, along which we stack the independant batches
# - The third dimension is the feature dimension, along which are the features of the vector representing the words

# In our toy case, we have inputs and outputs containing 3 features (third dimension !)
# We created a sequence of 5 different inputs (first dimension !)
# We don't use batch (the second dimension will have one lement)

# We need an initial hidden state, of the right sizes for dimension 2/3, but with only one temporal element:
# Here, it is:
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
# Why do we create a tuple of two tensors ? Because we use LSTMs: remember that they use two sets of weights,
# and two hidden states (Hidden state, and Cell state).
# If you don't remember, read: https://colah.github.io/posts/2015-08-Understanding-LSTMs/
# If we used a classic RNN, we would simply have:
# hidden = torch.randn(1, 1, 3)

# The naive way of applying a lstm to inputs is to apply it one step at a time, and loop through the sequence
for i in inputs:
    # After each step, hidden contains the hidden states (remember, it's a tuple of two states).
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    
# Alternatively, we can do the entire sequence all at once.
# The first value returned by LSTM is all of the Hidden states throughout the sequence.
# The second is just the most recent Hidden state and Cell state (you can compare the values)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence, for each temporal step
# "hidden" will allow you to continue the sequence and backpropagate later, with another sequence
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # Re-initialize
out, hidden = lstm(inputs, hidden)
pp.pprint(out)
pp.pprint(hidden)

tensor([[[ 0.2373,  0.0900, -0.4898]],

        [[ 0.2338,  0.4996, -0.0943]],

        [[-0.0330,  0.1991, -0.2932]],

        [[ 0.1509,  0.3386, -0.3763]],

        [[ 0.2319,  0.0651, -0.2004]]], grad_fn=<StackBackward>)
(tensor([[[ 0.2319,  0.0651, -0.2004]]], grad_fn=<StackBackward>),
 tensor([[[ 0.3897,  0.1889, -0.2667]]], grad_fn=<StackBackward>))


### Creating our own LSTM Model

In [20]:
# Models are usually implemented as custom nn.Module subclass
# We need to redefine the __init__ method, which creates the object
# We also need to redefine the forward method, which transform the input into outputs
# We can also add any method that we need: here, in order to initiate weights in the model

class LSTMModel(nn.Module):
    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(LSTMModel, self).__init__()
        # Create a dropout object to use on layers for regularization
        self.drop = nn.Dropout(dropout)
        # Create an encoder - which is an embedding layer
        self.encoder = nn.Embedding(ntoken, ninp)
        # Create the LSTM layers - find out how to stack them !
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        # Create what we call the decoder: a linear transformation to map the hidden state into scores for all words in the vocabulary
        # (Note that the softmax application function will be applied out of the model)
        self.decoder = nn.Linear(nhid, ntoken)
        
        # Initialize non-reccurent weights 
        self.init_weights()

        self.ninp = ninp
        self.nhid = nhid
        self.nlayers = nlayers
        
    def init_weights(self):
        # Initialize the encoder and decoder weights with the uniform distribution
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def init_hidden(self, batch_size):
        # Initialize the hidden state and cell state to zero, with the right sizes
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, batch_size, self.nhid),
                weight.new_zeros(self.nlayers, batch_size, self.nhid))    

    def forward(self, input, hidden, return_h=False):
        # Process the input
        emb = self.drop(self.encoder(input))   
        
        # Apply the LSTMs
        output, hidden = self.rnn(emb, hidden)
        
        # Decode into scores
        output = self.drop(output)      
        decoded = self.decoder(output)
        return decoded, hidden

### Building the Model

In [21]:
# Set the random seed manually for reproducibility.
torch.manual_seed(1)

# If you have Cuda installed and a GPU available
cuda = False
if torch.cuda.is_available():
    if not cuda:
        print("WARNING: You have a CUDA device, so you should probably choose cuda = True")
        
device = torch.device("cuda" if cuda else "cpu")

In [22]:
embedding_size = 200
hidden_size = 200
layers = 2
dropout = 0.5

###############################################################################
# Build the model
###############################################################################

vocab_size = len(corpus.dictionary)
model = LSTMModel(vocab_size, embedding_size, hidden_size, layers, dropout).to(device)
params = list(model.parameters())
criterion = nn.CrossEntropyLoss()

In [23]:
lr = 30.0
optimizer = 'sgd'
wdecay = 1.2e-6
# For gradient clipping
clip = 0.5

if optimizer == 'sgd':
    optim = torch.optim.SGD(params, lr=lr, weight_decay=wdecay)
if optimizer == 'adam':
    optim = torch.optim.Adam(params, lr=lr, weight_decay=wdecay)

In [24]:
# Let's think about gradient propagation:
# We plan to keep the second ouput of the LSTM layer (the hidden/cell states) to initialize
# the next call to LSTM. In this way, we can back-propagate the gradient for as long as we want.
# However, this put a huge strain on the memory used by the model, since it implies retaining
# a always-growing number of tensors of gradients in the cache.
# We decide to not backpropagate through time beyond the current sequence ! 
# We use a specific function to cut the 'hidden/state cell' states from their previous dependencies
# before using them to initialize the next call to the LSTM.
# This is done with the .detach() function.

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [25]:
# Other global parameters
epochs = 10
seq_len = 30
log_interval = 10
save = 'model.pt'

In [26]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, seq_len):
            data, targets = get_batch(data_source, i, seq_len)
            output, hidden = model(data, hidden)
            hidden = repackage_hidden(hidden)
            total_loss += len(data) * criterion(output.view(-1, vocab_size), targets.view(-1)).item()
    return total_loss / (len(data_source) - 1)

In [27]:
def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    hidden = model.init_hidden(batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, seq_len)):
        data, targets = get_batch(train_data, i, seq_len)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        optim.zero_grad()
        
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(params, clip)
        optim.step()
        
        total_loss += loss.data

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // seq_len, lr,
                elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [28]:
# Loop over epochs.
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

# Load the best saved model.
with open(save, 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    model.rnn.flatten_parameters()

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| epoch   1 |    10/  696 batches | lr 30.00 | ms/batch 2507.86 | loss 12.35 | ppl 230338.73
| epoch   1 |    20/  696 batches | lr 30.00 | ms/batch 1833.75 | loss  8.38 | ppl  4365.22
| epoch   1 |    30/  696 batches | lr 30.00 | ms/batch 1816.29 | loss  7.85 | ppl  2577.99
| epoch   1 |    40/  696 batches | lr 30.00 | ms/batch 1844.96 | loss  7.84 | ppl  2534.20
| epoch   1 |    50/  696 batches | lr 30.00 | ms/batch 1798.02 | loss  7.67 | ppl  2134.25
| epoch   1 |    60/  696 batches | lr 30.00 | ms/batch 1803.66 | loss  7.60 | ppl  1997.20
| epoch   1 |    70/  696 batches | lr 30.00 | ms/batch 1800.91 | loss  7.54 | ppl  1876.26
| epoch   1 |    80/  696 batches | lr 30.00 | ms/batch 1818.69 | loss  7.46 | ppl  1745.66
| epoch   1 |    90/  696 batches | lr 30.00 | ms/batch 1811.97 | loss  7.44 | ppl  1710.45
| epoch   1 |   100/  696 batches | lr 30.00 | ms/batch 1782.75 | loss  7.41 | ppl  1655.07
| epoch   1 |   110/  696 batches | lr 30.00 | ms/batch 1805.25 | loss  7.36 | 

/home/matthieu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTMModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/matthieu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/matthieu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/matthieu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name_

| epoch   2 |    10/  696 batches | lr 30.00 | ms/batch 1984.07 | loss  6.88 | ppl   969.45
| epoch   2 |    20/  696 batches | lr 30.00 | ms/batch 1801.44 | loss  6.30 | ppl   543.25
| epoch   2 |    30/  696 batches | lr 30.00 | ms/batch 1824.87 | loss  6.29 | ppl   540.36
| epoch   2 |    40/  696 batches | lr 30.00 | ms/batch 1807.56 | loss  6.32 | ppl   553.02
| epoch   2 |    50/  696 batches | lr 30.00 | ms/batch 1796.82 | loss  6.29 | ppl   541.73
| epoch   2 |    60/  696 batches | lr 30.00 | ms/batch 1798.36 | loss  6.30 | ppl   544.59
| epoch   2 |    70/  696 batches | lr 30.00 | ms/batch 1807.02 | loss  6.26 | ppl   520.75
| epoch   2 |    80/  696 batches | lr 30.00 | ms/batch 1800.52 | loss  6.32 | ppl   553.59
| epoch   2 |    90/  696 batches | lr 30.00 | ms/batch 1800.40 | loss  6.26 | ppl   523.03
| epoch   2 |   100/  696 batches | lr 30.00 | ms/batch 1805.30 | loss  6.29 | ppl   538.37
| epoch   2 |   110/  696 batches | lr 30.00 | ms/batch 1813.66 | loss  6.28 | p

| epoch   3 |   190/  696 batches | lr 30.00 | ms/batch 1813.06 | loss  5.92 | ppl   373.02
| epoch   3 |   200/  696 batches | lr 30.00 | ms/batch 1789.75 | loss  5.91 | ppl   368.29
| epoch   3 |   210/  696 batches | lr 30.00 | ms/batch 1804.42 | loss  5.88 | ppl   357.87
| epoch   3 |   220/  696 batches | lr 30.00 | ms/batch 1820.85 | loss  5.85 | ppl   345.79
| epoch   3 |   230/  696 batches | lr 30.00 | ms/batch 1798.36 | loss  5.94 | ppl   379.29
| epoch   3 |   240/  696 batches | lr 30.00 | ms/batch 1789.07 | loss  5.87 | ppl   353.53
| epoch   3 |   250/  696 batches | lr 30.00 | ms/batch 1814.45 | loss  5.87 | ppl   354.57
| epoch   3 |   260/  696 batches | lr 30.00 | ms/batch 1813.24 | loss  5.80 | ppl   331.51
| epoch   3 |   270/  696 batches | lr 30.00 | ms/batch 1786.96 | loss  5.88 | ppl   358.86
| epoch   3 |   280/  696 batches | lr 30.00 | ms/batch 1796.88 | loss  5.83 | ppl   342.02
| epoch   3 |   290/  696 batches | lr 30.00 | ms/batch 1836.86 | loss  5.89 | p

| epoch   4 |   370/  696 batches | lr 30.00 | ms/batch 1440.35 | loss  5.68 | ppl   291.96
| epoch   4 |   380/  696 batches | lr 30.00 | ms/batch 1489.61 | loss  5.65 | ppl   282.95
| epoch   4 |   390/  696 batches | lr 30.00 | ms/batch 1438.60 | loss  5.63 | ppl   279.16
| epoch   4 |   400/  696 batches | lr 30.00 | ms/batch 1627.61 | loss  5.62 | ppl   274.92
| epoch   4 |   410/  696 batches | lr 30.00 | ms/batch 1517.17 | loss  5.68 | ppl   292.44
| epoch   4 |   420/  696 batches | lr 30.00 | ms/batch 1414.31 | loss  5.62 | ppl   274.76
| epoch   4 |   430/  696 batches | lr 30.00 | ms/batch 1435.17 | loss  5.62 | ppl   275.88
| epoch   4 |   440/  696 batches | lr 30.00 | ms/batch 1437.49 | loss  5.59 | ppl   268.83
| epoch   4 |   450/  696 batches | lr 30.00 | ms/batch 1458.02 | loss  5.67 | ppl   291.25
| epoch   4 |   460/  696 batches | lr 30.00 | ms/batch 1501.95 | loss  5.61 | ppl   272.87
| epoch   4 |   470/  696 batches | lr 30.00 | ms/batch 1502.44 | loss  5.64 | p

| epoch   5 |   550/  696 batches | lr 30.00 | ms/batch 1435.01 | loss  5.48 | ppl   238.82
| epoch   5 |   560/  696 batches | lr 30.00 | ms/batch 1506.47 | loss  5.51 | ppl   246.85
| epoch   5 |   570/  696 batches | lr 30.00 | ms/batch 1435.58 | loss  5.49 | ppl   241.58
| epoch   5 |   580/  696 batches | lr 30.00 | ms/batch 1440.36 | loss  5.47 | ppl   238.49
| epoch   5 |   590/  696 batches | lr 30.00 | ms/batch 1438.27 | loss  5.49 | ppl   241.30
| epoch   5 |   600/  696 batches | lr 30.00 | ms/batch 1417.74 | loss  5.47 | ppl   238.09
| epoch   5 |   610/  696 batches | lr 30.00 | ms/batch 1458.83 | loss  5.39 | ppl   220.03
| epoch   5 |   620/  696 batches | lr 30.00 | ms/batch 1448.79 | loss  5.48 | ppl   239.73
| epoch   5 |   630/  696 batches | lr 30.00 | ms/batch 1434.98 | loss  5.49 | ppl   243.19
| epoch   5 |   640/  696 batches | lr 30.00 | ms/batch 1445.44 | loss  5.45 | ppl   233.06
| epoch   5 |   650/  696 batches | lr 30.00 | ms/batch 1449.84 | loss  5.48 | p

| epoch   7 |    10/  696 batches | lr 30.00 | ms/batch 1641.81 | loss  6.00 | ppl   404.63
| epoch   7 |    20/  696 batches | lr 30.00 | ms/batch 1530.21 | loss  5.43 | ppl   227.93
| epoch   7 |    30/  696 batches | lr 30.00 | ms/batch 1457.14 | loss  5.41 | ppl   223.98
| epoch   7 |    40/  696 batches | lr 30.00 | ms/batch 1486.60 | loss  5.46 | ppl   234.64
| epoch   7 |    50/  696 batches | lr 30.00 | ms/batch 1502.50 | loss  5.40 | ppl   221.22
| epoch   7 |    60/  696 batches | lr 30.00 | ms/batch 1467.79 | loss  5.39 | ppl   220.04
| epoch   7 |    70/  696 batches | lr 30.00 | ms/batch 1514.75 | loss  5.37 | ppl   214.13
| epoch   7 |    80/  696 batches | lr 30.00 | ms/batch 1468.08 | loss  5.45 | ppl   232.13
| epoch   7 |    90/  696 batches | lr 30.00 | ms/batch 1476.05 | loss  5.43 | ppl   228.61
| epoch   7 |   100/  696 batches | lr 30.00 | ms/batch 1542.29 | loss  5.46 | ppl   234.64
| epoch   7 |   110/  696 batches | lr 30.00 | ms/batch 1493.10 | loss  5.42 | p

| epoch   8 |   190/  696 batches | lr 30.00 | ms/batch 1470.94 | loss  5.36 | ppl   212.33
| epoch   8 |   200/  696 batches | lr 30.00 | ms/batch 1529.78 | loss  5.37 | ppl   214.23
| epoch   8 |   210/  696 batches | lr 30.00 | ms/batch 1500.65 | loss  5.35 | ppl   211.25
| epoch   8 |   220/  696 batches | lr 30.00 | ms/batch 1503.32 | loss  5.32 | ppl   205.15
| epoch   8 |   230/  696 batches | lr 30.00 | ms/batch 1488.79 | loss  5.39 | ppl   218.50
| epoch   8 |   240/  696 batches | lr 30.00 | ms/batch 1496.00 | loss  5.34 | ppl   207.60
| epoch   8 |   250/  696 batches | lr 30.00 | ms/batch 1506.65 | loss  5.33 | ppl   205.98
| epoch   8 |   260/  696 batches | lr 30.00 | ms/batch 1636.54 | loss  5.29 | ppl   198.04
| epoch   8 |   270/  696 batches | lr 30.00 | ms/batch 1445.35 | loss  5.38 | ppl   217.56
| epoch   8 |   280/  696 batches | lr 30.00 | ms/batch 1452.19 | loss  5.31 | ppl   203.09
| epoch   8 |   290/  696 batches | lr 30.00 | ms/batch 1469.00 | loss  5.40 | p

| epoch   9 |   370/  696 batches | lr 30.00 | ms/batch 1905.94 | loss  5.28 | ppl   195.75
| epoch   9 |   380/  696 batches | lr 30.00 | ms/batch 1593.92 | loss  5.24 | ppl   189.61
| epoch   9 |   390/  696 batches | lr 30.00 | ms/batch 1477.15 | loss  5.23 | ppl   186.80
| epoch   9 |   400/  696 batches | lr 30.00 | ms/batch 1436.77 | loss  5.26 | ppl   191.86
| epoch   9 |   410/  696 batches | lr 30.00 | ms/batch 1476.38 | loss  5.29 | ppl   197.78
| epoch   9 |   420/  696 batches | lr 30.00 | ms/batch 1419.62 | loss  5.25 | ppl   189.67
| epoch   9 |   430/  696 batches | lr 30.00 | ms/batch 1436.09 | loss  5.24 | ppl   187.90
| epoch   9 |   440/  696 batches | lr 30.00 | ms/batch 1430.02 | loss  5.21 | ppl   183.23
| epoch   9 |   450/  696 batches | lr 30.00 | ms/batch 1452.70 | loss  5.30 | ppl   200.44
| epoch   9 |   460/  696 batches | lr 30.00 | ms/batch 1431.42 | loss  5.23 | ppl   186.86
| epoch   9 |   470/  696 batches | lr 30.00 | ms/batch 1428.81 | loss  5.26 | p

| epoch  10 |   550/  696 batches | lr 30.00 | ms/batch 1428.27 | loss  5.18 | ppl   177.19
| epoch  10 |   560/  696 batches | lr 30.00 | ms/batch 1406.93 | loss  5.21 | ppl   182.51
| epoch  10 |   570/  696 batches | lr 30.00 | ms/batch 1449.27 | loss  5.18 | ppl   178.32
| epoch  10 |   580/  696 batches | lr 30.00 | ms/batch 1575.68 | loss  5.16 | ppl   173.67
| epoch  10 |   590/  696 batches | lr 30.00 | ms/batch 1485.45 | loss  5.19 | ppl   179.19
| epoch  10 |   600/  696 batches | lr 30.00 | ms/batch 1503.86 | loss  5.16 | ppl   173.94
| epoch  10 |   610/  696 batches | lr 30.00 | ms/batch 1462.33 | loss  5.09 | ppl   162.76
| epoch  10 |   620/  696 batches | lr 30.00 | ms/batch 1512.92 | loss  5.17 | ppl   175.91
| epoch  10 |   630/  696 batches | lr 30.00 | ms/batch 1488.89 | loss  5.19 | ppl   180.03
| epoch  10 |   640/  696 batches | lr 30.00 | ms/batch 1448.40 | loss  5.16 | ppl   174.23
| epoch  10 |   650/  696 batches | lr 30.00 | ms/batch 1439.75 | loss  5.18 | p

In [86]:
text = "What do you think about"

# Transform new text into a tensor to give as input to the model
words = text.split()
ids = torch.LongTensor(len(words))
for token, word in enumerate(words):
    ids[token] = corpus.dictionary.word2idx[word]

In [87]:
# We want the tensor to have the same dimensions as before : (sequence_len, batch_size)
ids = ids.unsqueeze(-1)
print(ids)

tensor([[4057],
        [ 421],
        [4481],
        [6014],
        [ 722]])


In [88]:
# Choose the number of generated words, the temperature, the top-k words to sample from, and initiate the hidden state
n_gen = 15
temperature = 0.7
k = 200
hidden = model.init_hidden(ids.shape[1])

for i in range(n_gen):
    output, hidden = model(ids, hidden)
    word_scores = output.squeeze().data.div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_scores[:k], 1)[0]
    ids = torch.cat((ids, word_idx.unsqueeze(-1)), dim=0)
    word = corpus.dictionary.idx2word[word_idx]

print('Generated text: ' + ' '.join([corpus.dictionary.idx2word[idx] for idx in ids]))

Generated text: What do you think about was and 's have the have 's had to have you 's have of .
